# Covid19dynstat - JupyterDash

The `jupyter-dash` package makes it easy to develop Plotly Dash apps from the Jupyter Notebook and JupyterLab.
Just replace the standard `dash.Dash` class with the `jupyter_dash.JupyterDash` subclass.

In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_player
import pandas as pd

When running in JupyterHub (or Binder), call the `infer_jupyter_config` function to detect the proxy configuration. This will detect the proper request_pathname_prefix and server_url values to use when displaying Dash apps. For example:  
server_url = `https://jupyter-jsc.fz-juelich.de`  
request_pathname_prefix = `/user/j.goebbert@fz-juelich.de/jureca_login/`  
For details please check the source here https://github.com/plotly/jupyter-dash/blob/v0.2.1.post1/jupyter_dash/comms.py#L33

In [6]:
JupyterDash.infer_jupyter_proxy_config()

**Attention:** I have to run this cell twice: first press play, wait a bit and hit play again while it still shows `[*]`

#### Create a Dash Flask server
Requests the browser to load Bootstrap 

In [8]:
# select a theme
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# external_stylesheets=[dbc.themes.BOOTSTRAP] -> default theme
# external_stylesheets=[dbc.themes.CYBORG]    -> dark theme

app.title = 'Covid-19-Interaktionsmodell'

# start the server
server = app.server
#print(app.get_asset_url('aaa'))

In [9]:
#import base64
from textwrap import dedent
from datetime import datetime as dt, timedelta
from dash.dependencies import Input, Output, State

asset_url=app.get_asset_url('assets') # = $JUPYTERHUB_SERVICE_PREFIX/assets
# print(asset_url)
# example: "https://jupyter-jsc.fz-juelich.de/user/<user>@fz-juelich.de/<machine>/proxy/8050/assets/"

metadata = pd.read_csv("assets/metadata.csv")

min_date=dt(2020, 1, 29).date()
max_date=dt(2020, 6, 16).date() # dt.today().date()
init_date=dt(2020, 6, 16).date() # dt.today().date()
init_countyid=1001
deltadays = 26

def get_assets_dir(date):
    date = dt.strptime(date.split(' ')[0], '%Y-%m-%d')
    assets_dir = (date -timedelta(days=deltadays)).strftime('%Y_%m_%d')
    return assets_dir

#### Define the top navigation bar

In [10]:
#####################
# Header and Footer
#####################
# https://dash-bootstrap-components.opensource.faculty.ai/docs/components/navbar/

navbar = dbc.NavbarSimple(
    brand="Bayessches räumlich-zeitliches Interaktionsmodell für Covid-19",
    brand_href="#",
    color="dark",
    fixed="top",
    dark=True,
    children=[
        dbc.NavItem(
            dbc.NavLink(
                "Artikel",
                href="https://nbviewer.jupyter.org/github/neuroinfo-os/BSTIM-Covid19/blob/master/notebooks/visualization_final.ipynb",
            )
        ),
        dbc.NavItem(
            dbc.NavLink(
                "Quellcode",
                href="https://github.com/neuroinfo-os/BSTIM-Covid19",
            )
        ),
    ])

navbar_footer = dbc.NavbarSimple(
    #brand="",
    brand_href="#",
    color="light",
    #fixed="bottom",
    #sticky=True,
    #dark=True,    
    children=[
        dbc.NavItem(
            dbc.NavLink(
                "Impressum",
                href="https://www.fz-juelich.de/portal/DE/Service/Impressum/impressum_node.html",
            )
        ),
        dbc.NavItem(
            dbc.NavLink(
                "Datenschutz",
                href="https://www.fz-juelich.de/portal/DE/datenschutz/_node.html",
            )
        ),
    ])

In [11]:
#####################
# Disclaimer
#####################
disclaimer_modal = html.Div(
    [
        dbc.Button("Disclaimer", id="disclaimer_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="disclaimer_modal",
            size="xl",
            children=[
                dbc.ModalHeader("Disclaimer"),
                dbc.ModalBody(
                    children=[
                        dcc.Markdown(
                            f"""
                            Für die Gesamtzahl der Infektionen pro Bundesland/Landkreis werden die den Gesundheitsämtern nach Infektionsschutzgesetz gemeldeten Fälle verwendet,
                            die dem RKI bis zum jeweiligen Tag um 0 Uhr übermittelt wurden.
                            Für die Analyse wird das Meldedatum verwendet, s. [Details zu den Daten](https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4)
                            Da es in dem Verfahren zu Differenzen zwischen Erkrankungsdatum und Meldedatum, sowie Verzögerungen in dem Meldeprozess geben kann,
                            ist die Analyse der Fallzahlen der letzten Woche bereits eine Vorhersage, die auf einer Schätzung basiert.
                            Alle hier präsentierten Ergebnisse basieren auf statistischen Methoden und bilden damit nicht das reale Geschehen, sondern Schätzungen ab, die von der wirklichen Situation abweichen können.
                            Dies ist bei der Interpretation der Ergebnisse zu berücksichtigen. 
                            Für eine detailliertere Analyse der COVID-19-Fälle verweisen wir auf den [täglichen Lagebericht des RKI](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Situationsberichte/Gesamt.html).
                            """
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="disclaimer_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
)
@app.callback(
    Output("disclaimer_modal", "is_open"),
    [Input("disclaimer_modal_open", "n_clicks"), Input("disclaimer_modal_close", "n_clicks")],
    [State("disclaimer_modal", "is_open")],
)
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open

In [12]:
#####################
# Date-Tabs (left)
#####################
left_date_tab1_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="left_date_tab1_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="left_date_tab1_modal",
            size="xl",
            children=[
                dbc.ModalHeader("Infektionen"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="left_date_modal1_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/map.png",
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="left_date_tab1_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
)
left_date_tab1 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="left_date_tab1_img_div",
                children=[
                    left_date_tab1_modal,
                    html.Img(
                        id="left_date_tab1_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/map.png",
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Die Infektionszahlen pro Tag (Welcher Tag (Nowcast ? Luke ) ) pro Landkreis und gewähltem Zeitfenster. "
                        "Der angezeigte Wert entspricht dem Nowcast, also der Schätzung der Anzahl der tatsächlich Neuinfizierten. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. ",
                        target="left_date_tab1_img",
                        style={"width": "200%"},
                        placement="left",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("left_date_tab1_modal", "is_open"),
    [Input("left_date_tab1_img_div", "n_clicks"), Input("left_date_tab1_modal_open", "n_clicks"), Input("left_date_tab1_modal_close", "n_clicks")],
    [State("left_date_tab1_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

#####################

left_date_tab2_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="left_date_tab2_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="left_date_tab2_modal",
            size="xl",
            children=[
                dbc.ModalHeader("Interaktionskernel"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="left_date_modal2_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/interaction_kernel.png",
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="left_date_tab2_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
) 
left_date_tab2 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",    
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="left_date_tab2_img_div",
                children=[
                    left_date_tab2_modal,
                    html.Img(
                        id="left_date_tab2_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/interaction_kernel.png",
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Der Interaktionskernel schätzt ab um wie stark eine gemeldete Infektion eine Neuansteckung in den nächsten Tagen "
                        "in einem Umkreis von bis zu 50km beeinflusst. "
                        "Diese Interaktion ist ein zusätzlicher Faktor der den Trend in einem Landkreis verstärkt oder abschwächt. "
                        "Eine warme Farbe indiziert, dass eine Covid-19 Meldung eine erhöhte Wahrscheinlichkeit einer Neuinfektion "
                        "im Verhältnis zum Trend zur Folge hat. "
                        "Eine starke Farben in der Nähe kleiner Radien bedeutet, dass das Infektionsgeschehen vor allem Auswirkungen "
                        "in der direkten Nähe der gemeldeten Fälle zur Folge hat. "
                        "Die Interaktion basiert auf einer Schätzung der Bevölkerungsdichte und der Form der Landkreise. "
                        "Daten zu den Wohnorten der Infizierten werden in dem Model nicht genutzt. "
                        "Alle hier genutzten Daten sind vollständig anonymisiert (siehe Erklärvideo). "
                        "Bei der Interpretation der Interaktionskernel ist dies zu berücksichtigen, und wir weisen darauf hin, dass dies nur eine Schätzung ist "
                        "die von der Realität abweichen kann.",
                        target="left_date_tab2_img",
                        style={"width": "200%"},
                        placement="left",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("left_date_tab2_modal", "is_open"),
    [Input("left_date_tab2_img_div", "n_clicks"), Input("left_date_tab2_modal_open", "n_clicks"), Input("left_date_tab2_modal_close", "n_clicks")],
    [State("left_date_tab2_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

In [13]:
#####################
# Date-Window Picker (left)
#####################
left_date_controls = dbc.FormGroup(
    children=[
        dbc.Label(
            id='left_date-label',
            children=["Datumsauswahl:"],
        ),
        html.Div(
            children=[
                dcc.DatePickerSingle(
                    id='left_date-picker',
                    display_format='DD. MMM YYYY',
                    min_date_allowed=min_date,
                    max_date_allowed=max_date,
                    initial_visible_month=init_date,
                    date=init_date,
                ),
                html.Div(
                    id='left_output-container-date-picker',
                    #style={'display': 'none'},
                    children=[(init_date -timedelta(days=deltadays)).strftime('%Y_%m_%d')],
                ),
            ]),
    ])

# Date Picker
@app.callback(
     Output(component_id='left_output-container-date-picker', component_property='children'),
    [Input(component_id='left_date-picker', component_property='date')])
def update_left_date_picker(date):
    if date is not None:
        return get_assets_dir(date)

# Map
@app.callback(
     Output(component_id='left_date_tab1_img', component_property='src'),
    [Input(component_id='left_date-picker', component_property='date')])
def update_left_date_tab1_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/map.png"
@app.callback(
     Output(component_id='left_date_modal1_img', component_property='src'),
    [Input(component_id='left_date-picker', component_property='date')])
def update_left_date_modal1_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/map.png"

# Interaction Kernel
@app.callback(
     Output(component_id='left_date_tab2_img', component_property='src'),
    [Input(component_id='left_date-picker', component_property='date')])
def update_left_date_tab2_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/interaction_kernel.png"
@app.callback(
     Output(component_id='left_date_modal2_img', component_property='src'),
    [Input(component_id='left_date-picker', component_property='date')])
def update_left_date_modal2_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/interaction_kernel.png"

In [14]:
#####################
# Date-Tabs (right)
#####################
right_date_tab1_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="right_date_tab1_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="right_date_tab1_modal",
            size="xl",
            children=[
                dbc.ModalHeader("Infektionen"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="right_date_modal1_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/map.png",
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="right_date_tab1_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
)
right_date_tab1 = dbc.Card(
    outline=True,
    color="light",    
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="right_date_tab1_img_div",
                children=[
                    right_date_tab1_modal,
                    html.Img(
                        id="right_date_tab1_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/map.png",
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Die Infektionszahlen pro Tag (Welcher Tag (Nowcast ? Luke ) ) pro Landkreis und gewählten Zeitfenster. "
                        "Der anzeigte Wert entspricht dem Nowcast, also der Schätzung der Anzahl der tatsächlich neuinfizierten. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. ",
                        target="right_date_tab1_img",
                        style={"width": "200%"},
                        placement="right",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("right_date_tab1_modal", "is_open"),
    [Input("right_date_tab1_img_div", "n_clicks"), Input("right_date_tab1_modal_open", "n_clicks"), Input("right_date_tab1_modal_close", "n_clicks")],
    [State("right_date_tab1_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

#####################

right_date_tab2_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="right_date_tab2_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="right_date_tab2_modal",
            size="xl",
            children=[
                dbc.ModalHeader("Interaktionskernel"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="right_date_modal2_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/interaction_kernel.png",
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="right_date_tab2_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
) 
right_date_tab2 = dbc.Card(
    outline=True,
    color="light",    
    className="mt-3",    
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="right_date_tab2_img_div",
                children=[
                    right_date_tab2_modal,
                    html.Img(
                        id="right_date_tab2_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/interaction_kernel.png",
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Der Interaktionskernel schätzt ab um wie stark eine gemeldete Infektion eine Neuansteckung in den nächsten Tagen "
                        "in einem Umkreis von bis zu 50km beeinflusst. "
                        "Diese Interaktion ist ein zusätzlicher Faktor der den Trend in einem Landkreis verstärkt oder abschwächt. "
                        "Eine warme Farbe indiziert, dass eine Covid-19 Meldung eine erhöhte Wahrscheinlichkeit einer Neuinfektion "
                        "im Verhältnis zum Trend zur Folge hat. "
                        "Eine starke Farben in der Nähe kleiner Radien bedeutet, dass das Infektionsgeschehen vor allem Auswirkungen "
                        "in der direkten Nähe der gemeldeten Fälle zur Folge hat. "
                        "Die Interaktion basiert auf einer Schätzung der Bevölkerungsdichte und der Form der Landkreise. "
                        "Daten zu den Wohnorten der Infizierten werden in dem Model nicht genutzt. "
                        "Alle hier genutzten Daten sind vollständig anonymisiert (siehe Erklärvideo). "
                        "Bei der Interpretation der Interaktionskernel ist dies zu berücksichtigen, und wir weisen darauf hin, dass dies nur eine Schätzung ist "
                        "die von der Realität abweichen kann.",
                        target="right_date_tab2_img",
                        style={"width": "200%"},
                        placement="right",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("right_date_tab2_modal", "is_open"),
    [Input("right_date_tab2_img_div", "n_clicks"), Input("right_date_tab2_modal_open", "n_clicks"), Input("right_date_tab2_modal_close", "n_clicks")],
    [State("right_date_tab2_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open


In [15]:
#####################
# Date-Window Picker (right)
#####################
right_date_controls = dbc.FormGroup(
    children=[
        dbc.Label(
            id='right_date-label',
            children=["Datumsauswahl:"],
        ),
        html.Div(
            children=[
                dcc.DatePickerSingle(
                    id='right_date-picker',
                    display_format='DD. MMM YYYY',
                    min_date_allowed=min_date,
                    max_date_allowed=max_date,
                    initial_visible_month=init_date,
                    date=init_date,
                ),
                html.Div(
                    id='right_output-container-date-picker',
                    #style={'display': 'none'},
                    children=[init_date.strftime('%Y_%m_%d')],
                ),
            ]),
    ])

# Date Picker
@app.callback(
     Output(component_id='right_output-container-date-picker', component_property='children'),
    [Input(component_id='right_date-picker', component_property='date')])
def update_right_date_picker(date):
    if date is not None:
        return get_assets_dir(date)

# Map
@app.callback(
     Output(component_id='right_date_tab1_img', component_property='src'),
    [Input(component_id='right_date-picker', component_property='date')])
def update_right_date_tab1_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/map.png"
@app.callback(
     Output(component_id='right_date_modal1_img', component_property='src'),
    [Input(component_id='right_date-picker', component_property='date')])
def update_right_date_modal1_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/map.png"

# Interaction Kernel
@app.callback(
     Output(component_id='right_date_tab2_img', component_property='src'),
    [Input(component_id='right_date-picker', component_property='date')])
def update_right_date_tab2_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/interaction_kernel.png"
@app.callback(
     Output(component_id='right_date_modal2_img', component_property='src'),
    [Input(component_id='right_date-picker', component_property='date')])
def update_right_date_modal2_img(date):
    if date is not None:
        assets_dir = get_assets_dir(date)
        return asset_url + assets_dir + "/interaction_kernel.png"

In [16]:
#####################
# County-Tabs (left)
#####################
left_pos_tab1_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="left_pos_tab1_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="left_pos_tab1_modal",
            size="xl",
            children=[
                dbc.ModalHeader("geglättet"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="left_pos_modal1_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_{0:05d}.png".format(init_countyid),
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="left_pos_tab1_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
) 
left_pos_tab1 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="left_pos_tab1_img_div",
                children=[
                    left_pos_tab1_modal,
                    html.Img(
                        id="left_pos_tab1_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_{0:05d}.png".format(init_countyid),
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Analyse und Vorhersage der Infektionszahlen für den ausgewählten Landkreis. "
                        "Der Nowcast entspricht der Schätzung der realen aktuellen Neuinfektionen für den angegebenden Tag. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. "
                        "Die Vorhersage nutzt das gleiche Modell um den Verlauf der kommenden 7 Tage, für die noch keine Zahlen vorliegen, vorherzusagen. "
                        "Das geglättete Model korrigiert die Ergebnisse bezüglich eines Wochenrhythmusses bei den Meldeverzögerungen (siehe Erklärvideo). ",
                        target="left_pos_tab1_img",
                        style={"width": "600px"},
                        placement="left",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("left_pos_tab1_modal", "is_open"),
    [Input("left_pos_tab1_img_div", "n_clicks"), Input("left_pos_tab1_modal_open", "n_clicks"), Input("left_pos_tab1_modal_close", "n_clicks")],
    [State("left_pos_tab1_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

#####################

left_pos_tab2_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="left_pos_tab2_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="left_pos_tab2_modal",
            size="xl",
            children=[
                dbc.ModalHeader("geglättet"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="left_pos_modal2_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_trend_{0:05d}.png".format(init_countyid),
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="left_pos_tab2_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
)
left_pos_tab2 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="left_pos_tab2_img_div",
                children=[
                    left_pos_tab2_modal,
                    html.Img(
                        id="left_pos_tab2_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_trend_{0:05d}.png".format(init_countyid),
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Analyse und Vorhersage der Infektionszahlen für den ausgewählten Landkreis. "
                        "Der Nowcast entspricht der Schätzung der realen aktuellen Neuinfektionen für den angegebenden Tag. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. "
                        "Die Vorhersage nutzt das gleiche Modell um den Verlauf der kommenden 7 Tage, für die noch keine Zahlen vorliegen, vorherzusagen. "
                        "Das geglättete Model korrigiert die Ergebnisse bezüglich eines Wochenrhythmusses bei den Meldeverzögerungen (siehe Erklärvideo). ",
                        target="left_pos_tab2_img",
                        style={"width": "200%"},
                        placement="left",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("left_pos_tab2_modal", "is_open"),
    [Input("left_pos_tab2_img_div", "n_clicks"), Input("left_pos_tab2_modal_open", "n_clicks"), Input("left_pos_tab2_modal_close", "n_clicks")],
    [State("left_pos_tab2_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

In [17]:
#####################
# County Picker (left)
#####################
left_pos_controls = dbc.FormGroup(
    children=[
        dbc.Label(
            id='left_pos-label',
            children=["Wähle Landkreis:"],
        ),
        html.Div(
            children=[
                dcc.Dropdown(
                    id="left_pos-variable",
                    value=init_countyid,
                    options=[
                        {"label": row['LKName'] + " (" + row['LKType'] + ")", "value": row['countyID']} for index, row in metadata.iterrows()
                    ]),
                html.Div(id='left_output-container-pos-variable'), #, style={'display': 'none'}),
            ]),  
    ])

# County Picker
@app.callback(
    Output(component_id='left_output-container-pos-variable', component_property='children'),
    [Input(component_id='left_pos-variable', component_property='value'),
     Input(component_id='left_output-container-date-picker', component_property='children')])
def update_left_pos_variable(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)

# geglättet
@app.callback(
     Output(component_id='left_pos_tab1_img', component_property='src'),
    [Input(component_id='left_pos-variable', component_property='value'),
     Input(component_id='left_output-container-date-picker', component_property='children')])
def update_left_pos_tab1_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)
@app.callback(
     Output(component_id='left_pos_modal1_img', component_property='src'),
    [Input(component_id='left_pos-variable', component_property='value'),
     Input(component_id='left_output-container-date-picker', component_property='children')])
def update_left_pos_modal1_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)

# ungeglättet
@app.callback(
     Output(component_id='left_pos_tab2_img', component_property='src'),
    [Input(component_id='left_pos-variable', component_property='value'),
     Input(component_id='left_output-container-date-picker', component_property='children')])
def update_left_pos_tab2_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_{0:05d}.png".format(value)
@app.callback(
     Output(component_id='left_pos_modal2_img', component_property='src'),
    [Input(component_id='left_pos-variable', component_property='value'),
     Input(component_id='left_output-container-date-picker', component_property='children')])
def update_left_pos_modal2_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_{0:05d}.png".format(value)

In [18]:
#####################
# County-Tabs (right)
#####################
right_pos_tab1_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="right_pos_tab1_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="right_pos_tab1_modal",
            size="xl",
            children=[
                dbc.ModalHeader("geglättet"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="right_pos_modal1_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_{0:05d}.png".format(init_countyid),
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="right_pos_tab1_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
) 
right_pos_tab1 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="right_pos_tab1_img_div",
                children=[
                    right_pos_tab1_modal,
                    html.Img(
                        id="right_pos_tab1_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_{0:05d}.png".format(init_countyid),
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Analyse und Vorhersage der Infektionszahlen für den ausgewählten Landkreis. "
                        "Der Nowcast entspricht der Schätzung der realen aktuellen Neuinfektionen für den angegebenden Tag. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. "
                        "Die Vorhersage nutzt das gleiche Modell um den Verlauf der kommenden 7 Tage, für die noch keine Zahlen vorliegen, vorherzusagen. "
                        "Das geglättete Model korrigiert die Ergebnisse bezüglich eines Wochenrhythmusses bei den Meldeverzögerungen (siehe Erklärvideo). ",
                        target="right_pos_tab1_img",
                        style={"width": "200%"},
                        placement="right",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("right_pos_tab1_modal", "is_open"),
    [Input("right_pos_tab1_img_div", "n_clicks"), Input("right_pos_tab1_modal_open", "n_clicks"), Input("right_pos_tab1_modal_close", "n_clicks")],
    [State("right_pos_tab1_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

#####################

right_pos_tab2_modal = html.Div(
    [
        dbc.Button("Vergrößern", id="right_pos_tab2_modal_open", outline=True, color="secondary", className="mr-1"),
        dbc.Modal(
            id="right_pos_tab2_modal",
            size="xl",
            children=[
                dbc.ModalHeader("geglättet"),
                dbc.ModalBody(
                    children=[
                        html.Img(
                            id="right_pos_modal2_img",
                            src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_trend_{0:05d}.png".format(init_countyid),
                            style={'width':'100%', 'height':'100%'},
                        ),
                    ]
                ),
                dbc.ModalFooter(
                    dbc.Button("Schließen", id="right_pos_tab2_modal_close", className="ml-auto")
                ),
            ],
        ),
    ]
)
right_pos_tab2 = dbc.Card(
    outline=True,
    color="light",
    className="mt-3",
    children=[ dbc.CardBody(
        children=[
            html.Div(
                id="right_pos_tab2_img_div",
                children=[
                    right_pos_tab2_modal,
                    html.Img(
                        id="right_pos_tab2_img",
                        src=asset_url + init_date.strftime('%Y_%m_%d') + "/curve_trend_{0:05d}.png".format(init_countyid),
                        style={'width':'100%', 'height':'100%'},
                    ),
                    dbc.Tooltip(
                        "Analyse und Vorhersage der Infektionszahlen für den ausgewählten Landkreis. "
                        "Der Nowcast entspricht der Schätzung der realen aktuellen Neuinfektionen für den angegebenden Tag. "
                        "Diese Schätzung korrigiert die gemeldeten Zahlen, die aufgrund von Verzögerungen im Meldeprozess "
                        "und einem unbekannten Erkrankungsdatum kleiner als die tatsächlichen Zahlen sein können, auf der Basis einer Vorhersage. "
                        "Die Vorhersage nutzt das gleiche Modell um den Verlauf der kommenden 7 Tage, für die noch keine Zahlen vorliegen, vorherzusagen. "
                        "Das geglättete Model korrigiert die Ergebnisse bezüglich eines Wochenrhythmusses bei den Meldeverzögerungen (siehe Erklärvideo). ",
                        target="right_pos_tab2_img",
                        style={"width": "200%"},
                        placement="right",
                    ),
                ]),
        ]),
    ])
@app.callback(
    Output("right_pos_tab2_modal", "is_open"),
    [Input("right_pos_tab2_img", "n_clicks"), Input("right_pos_tab2_modal_open", "n_clicks"), Input("right_pos_tab2_modal_close", "n_clicks")],
    [State("right_pos_tab2_modal", "is_open")],
)
def toggle_modal(n1, n2, n3, is_open):
    if n1 or n2 or n3:
        return not is_open
    return is_open

In [19]:
#####################
# County Picker (right)
#####################
right_pos_controls = dbc.FormGroup(
    children=[
        dbc.Label(
            id='right_pos-label',
            children=["Wähle Landkreis:"],
        ),
        html.Div(
            children=[
                dcc.Dropdown(
                    id="right_pos-variable",
                    value=init_countyid,
                    options=[
                        {"label": row['LKName'] + " (" + row['LKType'] + ")", "value": row['countyID']} for index, row in metadata.iterrows()
                    ]),
                html.Div(id='right_output-container-pos-variable'), #, style={'display': 'none'}),
            ]),  
    ])

# County Picker
@app.callback(
    Output(component_id='right_output-container-pos-variable', component_property='children'),
    [Input(component_id='right_pos-variable', component_property='value'),
     Input(component_id='right_output-container-date-picker', component_property='children')])
def update_right_pos_variable(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)

# geglättet
@app.callback(
     Output(component_id='right_pos_tab1_img', component_property='src'),
    [Input(component_id='right_pos-variable', component_property='value'),
     Input(component_id='right_output-container-date-picker', component_property='children')])
def update_right_pos_tab1_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)
@app.callback(
     Output(component_id='right_pos_modal1_img', component_property='src'),
    [Input(component_id='right_pos-variable', component_property='value'),
     Input(component_id='right_output-container-date-picker', component_property='children')])
def update_right_pos_modal1_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_trend_{0:05d}.png".format(value)

# ungeglättet
@app.callback(
     Output(component_id='right_pos_tab2_img', component_property='src'),
    [Input(component_id='right_pos-variable', component_property='value'),
     Input(component_id='right_output-container-date-picker', component_property='children')])
def update_right_pos_tab2_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_{0:05d}.png".format(value)
@app.callback(
     Output(component_id='right_pos_modal2_img', component_property='src'),
    [Input(component_id='right_pos-variable', component_property='value'),
     Input(component_id='right_output-container-date-picker', component_property='children')])
def update_right_pos_modal2_img(value, assets_dir):
    if value is not None:
        return asset_url + assets_dir + "/" + "curve_{0:05d}.png".format(value)

#### Define the main body of the webpage  
https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/  
Layout in Bootstrap is controlled using the grid system.
The Bootstrap grid has **twelve** columns, and **five** responsive tiers (allowing you to specify different behaviours on different screen sizes, see below).

In [20]:
#####################
# Main Structure
#####################
tab_height = '5vh'
body_layout = dbc.Container(
    style={"marginTop": 100},
    #fluid=True,
    children=[
        
        #####################
        # Introduction
        #####################
        
        dbc.Row(
            children=[
                dbc.Col(
                    style={
                        "marginBottom": 10,
                        "width": 12,
                    },                    
                    children=[
                        dcc.Markdown(
                            f"""
                            #####  **Ein Gemeinschaftsprojekt der Arbeitsgruppe [Neuroinformatik an der Universität Osnabrück](https://www.ikw.uni-osnabrueck.de/en/research_groups/neuroinformatics/people/prof_dr_gordon_pipa.html)**  
                            #####  **und des [Jülich Supercomputing Centre](https://www.fz-juelich.de/jsc), auf Basis der Daten des [RKI](https://www.rki.de/DE/Content/Infekt/IfSG/Signale/Projekte/Signale_Projekte_node.html;jsessionid=C61DE534E8208B0D69BEAD299FC753F9.internet091)**
                            """
                        ),
                    ]),                
            ]),
        dbc.Row(
            children=[
                dbc.Col(
                    width=4,
                    children=[
                        html.Img(
                            src='https://www.ikw.uni-osnabrueck.de/fileadmin/templates_global/public/img/header_logo.gif',
                            height='48', # width='500',
                            style={
                                'display':'block',
                                'margin-left': 'auto',
                                'margin-right': 'auto'
                            },
                        ),
                    ]),
                dbc.Col(
                    width=4,
                    children=[
 #                       html.Img(
 #                           src='https://www.rki.de/SiteGlobals/StyleBundles/Bilder/Farbschema_A/logo_a.jpg?__blob=normal&v=7',
 #                           height='48', # width='500',
 #                           style={
 #                               'display':'block',
 #                               'margin-left': 'auto',
 #                               'margin-right': 'auto'
 #                           },
 #                       ),
                    ]),                
                dbc.Col(
                    width=4,
                    children=[
                        html.Img(
                            src='https://www.vi-hps.org/cms/upload/logos/full/jsc-logo.png',
                            height='48', # width='500',
                            style={
                                'display':'block',
                                'margin-left': 'auto',
                                'margin-right': 'auto'
                            },
                        ),
                    ]),
            ]),
        dbc.Row(
            children=[
                dbc.Col(
                    style={
                        "marginTop": 30,
                        "width": 6,
                    },
                    children=[
                        dcc.Markdown(
                            f"""
                            -----
                            #####  BSTIM-Covid19 
                            -----
                            Aktuelle Daten und Vorhersage der Neuinfizierungen mit COVID-19 für Landkreise in Deutschland.
                            Das Model beschreibt die zeitliche Entwicklung der Neuinfizierungen in einen Zeitraum von 4 Wochen.
                            Das Model  beschreibt dazu nicht nur die wahrscheinlichste Entwicklung oder die mittlere Entwicklung,
                            sondern schätzt die Wahrscheinlichkeit für verschiedene Szenarien ab, die mit der aktuellen Datenlage kompatibel sind.
                            Zudem wir der Interaktionsradius vom Infektionsgeschehen geschätzt und als Interaktionskernel dargestellt.
                            Die Arbeit basiert auf einer Adaption des [BSTIM Models](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225838#pone.0225838.ref009) angepasst an die COVID-19 Situation.
                            Das Model beschreibt die tagesaktuellen Daten basierend auf den [Daten](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0/data?orderBy=Meldedatum) des RKI.
                            """
                        ),
                        disclaimer_modal,
                    ]),
                dbc.Col(
                    style={
                        "marginTop": 30,
                        "width": 6,
                    },
                    children=[
                        dcc.Markdown(
                            f"""
                            -----
                            #####  Wie funktioniert die Vorhersage und Analyse
                            -----
                            """
                        ),
                        html.Div(
                            style={
                                'width': '100%',
                                'float': 'left',
                                'margin': '0% 0% 5% 0%' # top, right, bottom, left
                            },
                            children=[
                                dash_player.DashPlayer(
                                    id='video-player',
                                    url='https://youtu.be/8-AfYeosBW8',
                                    controls=True,
                                    width='100%'
                                ),
                            ]), 
                    ]),
            ]),
        
        #####################
        # Plots Section
        #####################        
        
        dbc.Row(
            children=[
                dbc.Col(
                    dbc.Alert("Basisauswahl", color="primary")
                ),
                dbc.Col(
                    dbc.Alert("Vergleichsauswahl", color="primary")
                ),
            ]
        ),

        dbc.Row(
            children=[
                
                ##### left plots
                dbc.Col(
                    children=[
                        dbc.Card(
                            style={
                                'margin': '0% 0% 0% 0%', # top, right, bottom, left
                                'padding': '0',
                            },
                            body=True,                            
                            children=[
                                
                                # --- Zeitangabe (left) ---
                                dbc.CardHeader(
                                    left_date_controls,
                                ),
                                dbc.CardBody(
                                    className="mt-3",
                                    children=[
                                        dbc.Tabs(
                                            id="left_date-card-tabs",
                                            active_tab="tab-0",                                            
                                            children=[
                                                dbc.Tab(left_date_tab1, label="Infektionen",        style={'padding': '0', 'height': '550px'}),
                                                dbc.Tab(left_date_tab2, label="Interaktionskernel", style={'padding': '0', 'height': '550px'}),
                                            ]),
                                    
                                        html.P(
                                            id="left_pos-card-separator",
                                            className="card-text",
                                        ),
                                        
                                # --- Ortsangabe (left) ---
                                        dbc.Card(
                                            style={
                                                'margin': '0% 0% 0% 0%', # top, right, bottom, left
                                                'padding': '0',
                                            },                                 
                                            children=[
                                                dbc.CardHeader(
                                                    left_pos_controls,
                                                ),
                                                dbc.CardBody(
                                                    className="mt-3",
                                                    children=[
                                                        dbc.Tabs(
                                                            id="left_pos-card-tabs",
                                                            active_tab="tab-0",                                                   
                                                            children=[
                                                                dbc.Tab(left_pos_tab1, label="geglättet",    style={'padding': '0', 'height': '300px'}),
                                                                dbc.Tab(left_pos_tab2, label="ungeglättet",  style={'padding': '0', 'height': '300px'}),
                                                            ]),

                                                        html.P(
                                                            id="left_pos-card-content",
                                                            className="card-text",
                                                        ),
                                                    ]),
                                            ]),    
                                    ]),
                            ]),
                    ]),

                ##### right plots
                dbc.Col(
                    children=[
                        dbc.Card(
                            style={
                                'margin': '0% 0% 0% 0%', # top, right, bottom, left
                                'padding': '0',
                            },
                            body=True,                            
                            children=[
                                
                                # --- Zeitangabe (left) ---
                                dbc.CardHeader(
                                    right_date_controls,
                                ),
                                dbc.CardBody(
                                    className="mt-3",
                                    children=[
                                        dbc.Tabs(
                                            id="right_date-card-tabs",
                                            active_tab="tab-0",
                                            children=[
                                                dbc.Tab(right_date_tab1, label="Infektionen",        style={'padding': '0', 'height': '550px'}),
                                                dbc.Tab(right_date_tab2, label="Interaktionskernel", style={'padding': '0', 'height': '550px'}),
                                            ]),
                                    
                                        html.P(
                                            id="right_pos-card-separator",
                                            className="card-text",
                                        ),
                                        
                                # --- Ortsangabe (left) ---
                                        dbc.Card(
                                            style={
                                                'margin': '0% 0% 0% 0%', # top, right, bottom, left
                                                'padding': '0',
                                            },                                 
                                            children=[
                                                dbc.CardHeader(
                                                    right_pos_controls,
                                                ),
                                                dbc.CardBody(
                                                    className="mt-3",
                                                    children=[
                                                        dbc.Tabs(
                                                            id="right_pos-card-tabs",
                                                            active_tab="tab-0",                                                   
                                                            children=[
                                                                dbc.Tab(right_pos_tab1, label="geglättet",    style={'padding': '0', 'height': '300px'}),
                                                                dbc.Tab(right_pos_tab2, label="ungeglättet",  style={'padding': '0', 'height': '300px'}),
                                                            ]),

                                                        html.P(
                                                            id="right_pos-card-content",
                                                            className="card-text",
                                                        ),
                                                    ]),
                                            ]),    
                                    ]),
                            ]),
                    ]),
            ]),
    ])

app.layout = html.Div([navbar, body_layout, navbar_footer])

In [21]:
# multipage example: https://yadi.sk/d/JnM7BvKbJp3EdA
#@app.callback(Output('page-content', 'children'),
#              [Input('url', 'pathname')])
#def display_page(pathname):
#    if pathname == '/apps/app1':
#        return app1.layout()
#    else:
#        return app1.layout()
#
#@app.server.route('/static/<path:path>')
#def static_file(path):
#    static_folder = os.path.join(os.getcwd(), 'static')
#    return send_from_directory(static_folder, path)

#### Start the app

In [24]:
app.run_server(mode="external")
# mode="jupyterlab" -> will open the app in a tab in JupyterLab
# mode="inline"     -> will open the app below this cell
# mode="external"   -> will displays a URL that you can click on to open the app in a browser tab

Dash app running on https://jupyter-jsc.fz-juelich.de/user/j.goebbert@fz-juelich.de/juwels_login/proxy/8050/


--------------------------
**Attention**  
If you get the error "adress in use" this can also be the case because simply your layout has an error so that a dash-app could not been started. Open the app in a new browser-tab with the url
`<base-url>/proxy/<port>` where \<base-url\> derives from the url of your jupyterlab and \<port\> is by default 8050.  
For example: `https://jupyter-jsc.fz-juelich.de/user/j.goebbert@fz-juelich.de/jureca_login/proxy/8050`  
This will show the full error log.

--------------------------

Show the Dash Flask server is listening

In [ ]:
!echo "COMMAND     PID      USER   FD   TYPE   DEVICE SIZE/OFF NODE NAME"
!lsof -i -P -n | grep LISTEN